# Busca Informada (heurística)

Abordagem de busca que utiliza uma função heurística para definir qual sucessor é mais promissor para atingir uma meta (objetivo).

##### Heurística:
Função h(n) que estima o custo entre o estado atual e os estados gerados a partir dele

### Modelando o problema
- Sliding Puzzle
<img src="images/sliding_puzzle.gif" width="100" align="center">

#### Heurística: distância de Manhattan
$dist = |x_{1} - x_{2}| +  |y_{1} - y_{2}|$
<img src="images/manhattan_dist.png" width="300" align="center">


In [1]:
import numpy as np

class SlidingPuzzle():
    def __init__(self, num_blocos):
        '''
        Construtor
        Args:
            - num_blocos: numero de blocos por linha e coluna, valor inteiro (Ex: 3 significa 3 linhas e 3 colunas)
        '''
        self.num_blocos = num_blocos

    def _encontra_posicao(self, estado, elemento):
        '''
        Varre todo o tabuleiro (estado) e verifica em qual posição 'elemento' está
        Args:
            - estado: matriz contendo o estado do tabuleiro
            - elemento: elemento a ser buscado na matriz
        Return:
            - Retorna a linha e coluna onde o elemento se encontra
        '''
        for i in range(self.num_blocos):
            for j in range(self.num_blocos):
                if estado[i, j] == elemento:
                    return i, j
        return None, None

    def verifica_estados(self, atual, objetivo):
        '''
        Verifica se dois estados são iguais
        Args:
            - atual: matriz que descreve o estado atual
            - objetivo: matriz que descreve o estado objetivo
        Return:
            - booleano dizendo se o estado atual é ou não o objetivo
        '''
        flag = True
        for i in range(self.num_blocos):
            for j in range(self.num_blocos):
                if atual[i, j] != objetivo[i, j]:
                    flag = False
                    break

        return flag

    def expande_estados(self, atual):
        '''
        Dado o estado atual, realiza a expansão de estados
        Args:
            - atual: matriz que descreve o estado atual
        Return:
            - lista com os novos estados após a expansão
        '''
        
        novos_estados = []
        linha, coluna = self._encontra_posicao(atual, 0)

        # Cima
        if linha > 0:
            novo_estado = np.copy(atual)
            nova_linha = linha - 1

            bloco_alvo = novo_estado[nova_linha, coluna]
            novo_estado[nova_linha, coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)

        # Baixo
        if linha < self.num_blocos - 1:
            novo_estado = np.copy(atual)
            nova_linha = linha + 1

            bloco_alvo = novo_estado[nova_linha, coluna]
            novo_estado[nova_linha, coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)


        # Esquerda
        if coluna > 0:
            novo_estado = np.copy(atual)
            nova_coluna = coluna - 1

            bloco_alvo = novo_estado[linha, nova_coluna]
            novo_estado[linha, nova_coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)

        # Direita
        if coluna < self.num_blocos - 1:
            novo_estado = np.copy(atual)
            nova_coluna = coluna + 1

            bloco_alvo = novo_estado[linha, nova_coluna]
            novo_estado[linha, nova_coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)

        return novos_estados
    
    
    def distancia_Manhattan(self, atual, objetivo):
        '''
        Calcula a distância de Manhattan entre o estado atual e o estado objetivo
        Args: 
            - atual: estado atual do tabuleiro
            - objetivo: estado objetivo do problema
        Return:
            - Distância de Manhattan
        '''
        
        distancia_total = 0
        
        for i in range(self.num_blocos):
            for j in range(self.num_blocos):
                
                bloco_atual = atual[i, j]
                linha, coluna = self._encontra_posicao(objetivo, bloco_atual)
                
                distancia = abs(linha - i) + abs(coluna - j)
                distancia_total += distancia
                
        return distancia_total


### Busca Gulosa
Na busca gulosa o próximo estado mais próximo ao objetivo é visitado

In [2]:
# Fila de prioridade
import heapq

class BuscaGulosa():
    def __init__(self, problema):
        '''
        Construtor
        Args:
            - problema: objeto do problema a ser solucionado
        '''
        self.problema = problema
        
    def _verifica_visitado(self, estado, estados_visitados):
        '''
        Verifica se 'estado' está na lista de estados visitados
        Args:
            - estado: estado qualquer do tabuleiro
            - estados_visitados: lista com todos os estados já visitados
        Return:
            - booleano dizendo se o estado foi visitado ou não
        '''
        for i in estados_visitados:
            if self.problema.verifica_estados(i, estado):
                return True
        return False
    
    def busca(self, inicio, fim):
        '''
        Realiza a busca gulosa, armazenando os estados em uma FILA DE PRIORIDADES
        Args:
            - inicio: estado inicial do problema
            - fim: estado objetivo
        Return:
            - booleano se a solução foi encontrada, lista dos estados visitados, quantidade de estados visitados
        '''
        
        '''
        OBS.: A distância de Manhattan é inversamente proporcional à prioridade, quanto menor a distância, maior
        a prioridade desse estado
        '''
        
        p_fila = []
        
        # H, ID, elemento
        id_estado = 0
        heapq.heappush(p_fila, (0, id_estado, inicio))

        solucao_encontrada = False
        estados_visitados = []
        cont_estados = 0

        while not len(p_fila) == 0:
            atual = heapq.heappop(p_fila)[2]
            estados_visitados.append(atual)
            print(f"Estado:\n{atual}")

            if self.problema.verifica_estados(atual, fim):
                solucao_encontrada = True
                break
            else:
                cont_estados += 1
                print("Visitando #", cont_estados)
                novos_estados = self.problema.expande_estados(atual)
                print("Estados Gerados:")
                for i in novos_estados:
                    if not self._verifica_visitado(i, estados_visitados):
                        id_estado += 1
                        distancia = self.problema.distancia_Manhattan(i, fim)
                        print(i)
                        print(distancia)
                        heapq.heappush(p_fila, (distancia, id_estado, i))

        return solucao_encontrada, estados_visitados, cont_estados


### Execução

In [3]:
# Pacote auxiliar para o cálculo do tempo
from time import time

# Criando objeto do problema
problema = SlidingPuzzle(3)

# Criando Matriz inicial e matriz alvo
start = np.matrix([[1,0,2],[8,4,3],[7,6,5]])
target = np.matrix([[1,2,3],[8,0,4],[7,6,5]])

# Mostrando informações iniciais
print(f"Initial state: \n{start}")
print("*"*15)
print(f"Target state: \n{target}")
print("*"*15)

Initial state: 
[[1 0 2]
 [8 4 3]
 [7 6 5]]
***************
Target state: 
[[1 2 3]
 [8 0 4]
 [7 6 5]]
***************


In [4]:
# Execução do BFS
bg = BuscaGulosa(problema)

ini = time() # Tempo inicial

bg_solucao, bg_estados_visitados, bg_num_visitados = bg.busca(start, target) # chamando busca

bg_time = time()-ini # Tempo total

if bg_solucao:
    print(f"Solution found!!!")
else:
    print("Solution not found!!!")

Estado:
[[1 0 2]
 [8 4 3]
 [7 6 5]]
Visitando # 1
Estados Gerados:
[[1 4 2]
 [8 0 3]
 [7 6 5]]
4
[[0 1 2]
 [8 4 3]
 [7 6 5]]
6
[[1 2 0]
 [8 4 3]
 [7 6 5]]
4
Estado:
[[1 4 2]
 [8 0 3]
 [7 6 5]]
Visitando # 2
Estados Gerados:
[[1 4 2]
 [8 6 3]
 [7 0 5]]
6
[[1 4 2]
 [0 8 3]
 [7 6 5]]
6
[[1 4 2]
 [8 3 0]
 [7 6 5]]
6
Estado:
[[1 2 0]
 [8 4 3]
 [7 6 5]]
Visitando # 3
Estados Gerados:
[[1 2 3]
 [8 4 0]
 [7 6 5]]
2
Estado:
[[1 2 3]
 [8 4 0]
 [7 6 5]]
Visitando # 4
Estados Gerados:
[[1 2 3]
 [8 4 5]
 [7 6 0]]
4
[[1 2 3]
 [8 0 4]
 [7 6 5]]
0
Estado:
[[1 2 3]
 [8 0 4]
 [7 6 5]]
Solution found!!!


In [5]:
# Apresentando resultados
print("==== Busca Gulosa ====")
print(f"Solução encontrada? {bg_solucao}")
print(f"Número de estados visitados: {bg_num_visitados}")
print(f"Tempo de execução: {bg_time}")

==== Busca Gulosa ====
Solução encontrada? True
Número de estados visitados: 4
Tempo de execução: 0.008518695831298828


<img src="images/dist_manhattan.png" width="800" align="center">

### Resolvendo exercício da aula 6 utilizando busca gulosa

In [6]:
# Pacote auxiliar para o cálculo do tempo
from time import time

# Criando objeto do problema
problema = SlidingPuzzle(3)

# Criando Matriz inicial e matriz alvo
start = np.matrix([[2,8,3],[1,6,4],[7,0,5]])
target = np.matrix([[1,2,3],[8,0,4],[7,6,5]])

# Mostrando informações iniciais
print(f"Initial state: \n{start}")
print("*"*15)
print(f"Target state: \n{target}")
print("*"*15)

Initial state: 
[[2 8 3]
 [1 6 4]
 [7 0 5]]
***************
Target state: 
[[1 2 3]
 [8 0 4]
 [7 6 5]]
***************


In [7]:
# Execução do BFS
bg = BuscaGulosa(problema)

ini = time() # Tempo inicial

bg_solucao, bg_estados_visitados, bg_num_visitados = bg.busca(start, target) # chamando busca

bg_time = time()-ini # Tempo total

if bg_solucao:
    print(f"Solution found!!!")
else:
    print("Solution not found!!!")

Estado:
[[2 8 3]
 [1 6 4]
 [7 0 5]]
Visitando # 1
Estados Gerados:
[[2 8 3]
 [1 0 4]
 [7 6 5]]
4
[[2 8 3]
 [1 6 4]
 [0 7 5]]
8
[[2 8 3]
 [1 6 4]
 [7 5 0]]
8
Estado:
[[2 8 3]
 [1 0 4]
 [7 6 5]]
Visitando # 2
Estados Gerados:
[[2 0 3]
 [1 8 4]
 [7 6 5]]
4
[[2 8 3]
 [0 1 4]
 [7 6 5]]
6
[[2 8 3]
 [1 4 0]
 [7 6 5]]
6
Estado:
[[2 0 3]
 [1 8 4]
 [7 6 5]]
Visitando # 3
Estados Gerados:
[[0 2 3]
 [1 8 4]
 [7 6 5]]
4
[[2 3 0]
 [1 8 4]
 [7 6 5]]
6
Estado:
[[0 2 3]
 [1 8 4]
 [7 6 5]]
Visitando # 4
Estados Gerados:
[[1 2 3]
 [0 8 4]
 [7 6 5]]
2
Estado:
[[1 2 3]
 [0 8 4]
 [7 6 5]]
Visitando # 5
Estados Gerados:
[[1 2 3]
 [7 8 4]
 [0 6 5]]
4
[[1 2 3]
 [8 0 4]
 [7 6 5]]
0
Estado:
[[1 2 3]
 [8 0 4]
 [7 6 5]]
Solution found!!!


In [8]:
print(f"Estado encontrado? {bg_solucao}")
print(f"Numero de estados visitados{bg_num_visitados}")
print(f"Estados visitados:\n{bg_estados_visitados}")
print(f"Tempo: {bg_time}")

Estado encontrado? True
Numero de estados visitados5
Estados visitados:
[matrix([[2, 8, 3],
        [1, 6, 4],
        [7, 0, 5]]), array([[2, 8, 3],
       [1, 0, 4],
       [7, 6, 5]]), array([[2, 0, 3],
       [1, 8, 4],
       [7, 6, 5]]), array([[0, 2, 3],
       [1, 8, 4],
       [7, 6, 5]]), array([[1, 2, 3],
       [0, 8, 4],
       [7, 6, 5]]), array([[1, 2, 3],
       [8, 0, 4],
       [7, 6, 5]])]
Tempo: 0.008167743682861328
